In [ ]:
import sys
import pyaudio
import math
import struct
import time
import wave

In [ ]:
p = pyaudio.PyAudio()

In [ ]:
def callback(in_data, frame_count, time_info, status):
    floatLevels = []
    for _i in range(1024):
        floatLevels.append(struct.unpack('<h', in_data[_i:_i + 2])[0])
    avgChunk = sum(floatLevels)/len(floatLevels)
    print_audio_level(avgChunk, time_info['current_time'])
    return (in_data, pyaudio.paContinue)


time_difference = [0, 0]
def print_audio_level(in_data, callback_time):
    time_difference [1] = callback_time
    if time_difference [1] - time_difference [0] > 0.1:
        time_difference [0] = time_difference [1]
        level = get_level_dB(in_data)
        sys.stdout.write('Audio level' + str(level) + '\r')
        sys.stdout.flush()

def get_level_dB(sample_value):
    MAX_SAMPLE_VALUE = 32768
    try:
        level = (20 * math.log10(float(abs(sample_value)) / MAX_SAMPLE_VALUE))
    except ValueError:
        level = - 96.32
    return int(level)

In [ ]:
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=48000,
                frames_per_buffer=1024,
                input=True,
                output=True,
                stream_callback=callback)

# Start the stream
stream.start_stream()

# Close the stream after 10 seconds
time.sleep(10)

stream.close()

In [ ]:
form_1 = pyaudio.paInt16 # 16-bit resolution
chans = 1 # 1 channel
samp_rate = 44100 # 44.1kHz sampling rate
chunk = 4096 # 2^12 samples for buffer
record_secs = 5 # seconds to record
dev_index = 2 # device index found by p.get_device_info_by_index(ii)
wav_output_filename = 'output/myvoice.wav'

audio = pyaudio.PyAudio() # create pyaudio instantiation

# create pyaudio stream
stream = audio.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=samp_rate, 
                    #input_device_index=dev_index,
                    input=True, 
                    frames_per_buffer=chunk)

print("recording")
frames = []

# loop through stream and append audio chunks to frame array
for ii in range(0,int((samp_rate/chunk)*record_secs)):
    data = stream.read(chunk)
    frames.append(data)

print("finished recording")

# stop the stream, close it, and terminate the pyaudio instantiation
stream.stop_stream()
stream.close()
audio.terminate()

# save the audio frames as .wav file
wavefile = wave.open(wav_output_filename,'wb')
wavefile.setnchannels(chans)
wavefile.setsampwidth(audio.get_sample_size(form_1))
wavefile.setframerate(samp_rate)
wavefile.writeframes(b''.join(frames))
wavefile.close()